In [3]:
!pip install -q ultralytics opencv-python pillow pyyaml albumentations
print("Dependências instaladas")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.4 MB/s eta 0:00:00
Dependências instaladas


In [4]:
import torch
import time
import yaml
import cv2
import numpy as np
import shutil
from pathlib import Path
from ultralytics import YOLO

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PyTorch: 2.8.0+cu126
CUDA: True
GPU: NVIDIA A100-SXM4-40GB


In [5]:
SAVE_DIR = "/content/drive/MyDrive/PPE_Detection"
DATASET_BASE = "/content/datasets/ppe"
DATASET_AUG = "/content/datasets/ppe_augmented"

print(f"Diretório de modelos: {SAVE_DIR}")
print(f"Dataset base: {DATASET_BASE}")
print(f"Dataset aumentado: {DATASET_AUG}")

Diretório de modelos: /content/drive/MyDrive/PPE_Detection
Dataset base: /content/datasets/ppe
Dataset aumentado: /content/datasets/ppe_augmented


In [6]:
!pip install -q roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="tSGbJRy40ipX4Xo7rYhw")
project = rf.workspace("joseph-nelson").project("hard-hat-workers")
dataset = project.version(2).download("yolov8", location=DATASET_BASE)

train_imgs = len(list(Path(DATASET_BASE + '/train/images').glob('*.jpg')))
valid_imgs = len(list(Path(DATASET_BASE + '/valid/images').glob('*.jpg')))

print(f"Treino: {train_imgs} imagens")
print(f"Validacao: {valid_imgs} imagens")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 134.2 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/datasets/ppe in yolov8:: 100%|██████████| 14079/14079 [00:01<00:00, 9989.70it/s] 

Treino: 5269 imagens
Validacao: 0 imagens


In [7]:
import shutil

# Verifica estrutura
print("Estrutura atual:")
print(f"Treino: {len(list(Path(DATASET_BASE + '/train/images').glob('*.jpg')))} imgs")
print(f"Validacao: {len(list(Path(DATASET_BASE + '/valid/images').glob('*.jpg')))} imgs")

# Se validação está vazia, cria split
valid_dir = Path(DATASET_BASE) / 'valid'
if not (valid_dir / 'images').exists() or len(list((valid_dir / 'images').glob('*.jpg'))) == 0:
    print("\nCriando split de validacao...")

    (valid_dir / 'images').mkdir(parents=True, exist_ok=True)
    (valid_dir / 'labels').mkdir(parents=True, exist_ok=True)

    train_imgs = list(Path(DATASET_BASE + '/train/images').glob('*.jpg'))

    # Move 20% para validação
    n_valid = max(10, len(train_imgs) // 5)

    for img_path in train_imgs[:n_valid]:
        # Move imagem
        shutil.move(str(img_path), str(valid_dir / 'images' / img_path.name))

        # Move label
        label_path = Path(DATASET_BASE) / 'train' / 'labels' / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.move(str(label_path), str(valid_dir / 'labels' / label_path.name))

    print(f"Movidas {n_valid} imagens para validacao")

# Verifica final
train_final = len(list(Path(DATASET_BASE + '/train/images').glob('*.jpg')))
valid_final = len(list(Path(DATASET_BASE + '/valid/images').glob('*.jpg')))

print(f"\nFinal:")
print(f"Treino: {train_final} imgs")
print(f"Validacao: {valid_final} imgs")

Estrutura atual:
Treino: 5269 imgs
Validacao: 0 imgs

Criando split de validacao...
Movidas 1053 imagens para validacao

Final:
Treino: 4216 imgs
Validacao: 1053 imgs


In [9]:
# CÉLULA 5 OTIMIZADA: Treinamento A100
model = YOLO('yolov8n.pt')

results = model.train(
    data=DATASET_BASE + '/data.yaml',
    epochs=20,        # Reduzido
    imgsz=640,
    batch=64,         # Maior para A100
    device=0,
    workers=8,        # Otimizado
    cache=True,       # Cache em RAM
    project=SAVE_DIR,
    name='ppe_model',
    patience=8,
    save=True,
    exist_ok=True,
    verbose=False,    # Menos logging
    plots=False       # Sem plots (economiza tempo)
)

print(f"Modelo: {SAVE_DIR}/ppe_model/weights/best.pt")

Ultralytics 8.3.213 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/ppe/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ppe_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=8, perspective=0.0, plots

In [10]:
model_path = SAVE_DIR + '/ppe_model/weights/best.pt'
test_model = YOLO(model_path)

test_img = list(Path(DATASET_BASE + '/valid/images').glob('*.jpg'))[0]
results = test_model.predict(test_img, conf=0.3, verbose=False)
boxes = results[0].boxes

print(f"Imagem: {test_img.name}")
print(f"Detecções: {len(boxes)}")

for i, box in enumerate(boxes[:5]):
    cls_name = test_model.names[int(box.cls[0])]
    conf = float(box.conf[0])
    print(f"  {i+1}. {cls_name}: {conf:.1%}")

Imagem: 004988_jpg.rf.e34b8cefab46dbd23426077a03d21edc.jpg
Detecções: 3
  1. helmet: 87.7%
  2. head: 84.6%
  3. helmet: 43.1%


In [11]:
model = YOLO(model_path)
metrics = model.val(data=DATASET_BASE + '/data.yaml')

print(f"mAP50: {metrics.box.map50:.1%}")
print(f"Precision: {metrics.box.mp:.1%}")
print(f"Recall: {metrics.box.mr:.1%}")

Ultralytics 8.3.213 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1277.1±526.1 MB/s, size: 31.7 KB)
val: Scanning /content/datasets/ppe/valid/labels.cache... 1053 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1053/1053 1.7Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 66/66 10.7it/s 6.2s
                   all       1053       4136      0.961      0.615      0.652      0.455
                  head        211        995      0.932      0.915      0.953      0.664
                helmet        963       3048      0.953       0.93      0.977      0.686
                person         27         93          1          0     0.0275     0.0149
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/runs/dete

In [13]:
!pip install -q gradio

import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
from pathlib import Path

model_path = SAVE_DIR + '/ppe_model/weights/best.pt'
model_cache = YOLO(model_path)

def detect_ppe(image, confidence, show_labels, show_conf):
    img_array = np.array(image)
    results = model_cache.predict(img_array, conf=confidence, verbose=False)[0]
    boxes = results.boxes
    annotated = img_array.copy()

    colors = [(255,0,0), (0,255,0), (255,165,0), (255,255,0), (255,0,255),
              (0,255,255), (128,0,128), (0,128,0), (0,0,255), (128,128,128)]

    detections = {}

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        cls_name = model_cache.names[cls_id]
        color = colors[cls_id % len(colors)]

        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 3)

        if show_labels or show_conf:
            label = []
            if show_labels:
                label.append(cls_name)
            if show_conf:
                label.append(f"{conf:.0%}")
            text = " ".join(label)

            (w, h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(annotated, (x1, y1-25), (x1+w+10, y1), color, -1)
            cv2.putText(annotated, text, (x1+5, y1-8),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        if cls_name not in detections:
            detections[cls_name] = []
        detections[cls_name].append(conf)

    info = [f"Confianca minima: {confidence}", f"Deteccoes: {len(boxes)}", ""]

    if detections:
        info.append("Por classe:")
        for cls, confs in sorted(detections.items()):
            info.append(f"  {cls}: {len(confs)}x (media: {np.mean(confs):.1%})")
    else:
        info.append("Nenhuma deteccao")

    return Image.fromarray(annotated), "\n".join(info)

with gr.Blocks(title="Deteccao EPIs") as demo:
    gr.Markdown("# Deteccao de EPIs com YOLOv8")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Carregar Imagem")
            conf_slider = gr.Slider(0.1, 1.0, 0.4, 0.05, label="Confianca Minima")

            with gr.Row():
                labels_check = gr.Checkbox(value=True, label="Mostrar Labels")
                conf_check = gr.Checkbox(value=True, label="Mostrar Confianca")

            btn = gr.Button("Detectar EPIs", variant="primary", size="lg")

        with gr.Column():
            image_output = gr.Image(label="Resultado")
            text_output = gr.Textbox(label="Informacoes", lines=12)

    btn.click(
        detect_ppe,
        [image_input, conf_slider, labels_check, conf_check],
        [image_output, text_output]
    )

    gr.Markdown(f"""
    ### Modelo Treinado
    - Dataset: {len(list(Path(DATASET_BASE + '/train/images').glob('*.jpg')))} treino
    - Modelo: YOLOv8n
    - Classes: {len(model_cache.names)}
    """)

print("Iniciando Gradio...")
demo.launch(share=True, debug=False)

Iniciando Gradio...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4c20c1d5ebdde9ef7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
